## 什麼都不做，單純把缺值填入，看看預測率多少  
Try two measures of filling NA  
1.  
numeric -- fill in mean    
objective -- fill in mode  
2.  
numeric -- fill in mean  
objective -- fill in proportion  


In [20]:
import os
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from xgboost import XGBClassifier

train = pd.read_csv('train.csv',encoding = 'big5')
test = pd.read_csv('test.csv',encoding = 'big5')

/Users/liaoshousan/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (17,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
#Select objective data
train_obj = train.select_dtypes(include=['object'])
test_obj = test.select_dtypes(include=['object'])

train_obj

,GENDER,AGE,CHARGE_CITY_CD,CONTACT_CITY_CD,LAST_A_CCONTACT_DT,LAST_A_ISSUE_DT,LAST_B_ISSUE_DT,APC_1ST_AGE,INSD_1ST_AGE,IF_2ND_GEN_IND,...,IF_ADD_INSD_IND,CUST_9_SEGMENTS_CD,FINANCETOOLS_A,FINANCETOOLS_B,FINANCETOOLS_C,FINANCETOOLS_D,FINANCETOOLS_E,FINANCETOOLS_F,FINANCETOOLS_G,Y1
0,M,低,A1,A1,Y,N,N,低,低,N,...,N,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,M,低,A1,A1,Y,N,N,低,中,N,...,N,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,M,低,A1,A1,Y,Y,N,低,中,Y,...,N,A,Y,N,N,N,N,N,N,N
3,M,低,A1,A1,Y,N,N,低,中,N,...,N,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
4,M,低,A1,A1,N,N,N,低,低,Y,...,N,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,F,高,A1,A1,N,N,N,中高,NaN,N,...,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
99996,F,高,E,E,Y,N,N,中高,NaN,N,...,NaN,H,N,N,Y,N,N,N,N,Y
99997,F,中高,C2,C2,Y,N,N,中,NaN,N,...,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
99998,M,高,A1,A1,Y,Y,N,高,NaN,N,...,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N


In [22]:
# Observe unique value in train_obj
for i in train_obj.columns:
    print(train_obj[i].unique())


['M' 'F' nan]
['低' '中高' '中' '高']
['A1' 'A2' 'B1' 'B2' 'D' 'C1' 'C2' 'E']
['A1' 'A2' 'B1' 'B2' 'C1' 'D' 'C2' 'E']
['Y' 'N']
['N' 'Y']
['N' 'Y']
['低' '中' '中高' '高' nan]
['低' '中' '中高' '高' nan]
['N' 'Y']
['低' '中高' '中' '高' nan]
['低' '中' '中高' '高' nan]
['中' '低' '高']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['Y' 'N']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
[nan 'Y' 'N']
[nan 'N' 'Y']
[nan 'N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y']
['N' 'Y' nan]
['N' 'Y' nan]
['N' 'Y' nan]
['N' 'Y' nan]
['N' 'Y' nan]
['N' 'Y' nan]
['N' 'Y' nan]
['N' 'Y' nan]
['N' 'Y']
['N' nan 'Y']
['N' nan 'Y']
['N' nan 'Y']
['N' nan 'Y']
['N' nan 'Y']
['N' 'Y' nan]
['N' nan 'Y']
['N' nan 'Y']
['N' 'Y' nan]
['N' nan 'Y']
['N' nan 'Y']
['N' nan 'Y']
['N' nan 'Y']
['Y' 'N' nan]
['N' nan 'Y']
['N' 'Y' nan]
['N' 'Y' nan]
[nan 'Y' 

# Measure 1

### Objective --> Fillna(Mode)

In [23]:
for col in train_obj.columns:
    train_obj[col].fillna(train_obj[col].mode()[0],inplace=True)

for col in test_obj.columns:
    test_obj[col].fillna(train_obj[col].mode()[0],inplace=True)


/Users/liaoshousan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


### Label Encoding

In [24]:
train_obj = train_obj.apply(LabelEncoder().fit_transform)
test_obj = test_obj.apply(LabelEncoder().fit_transform)

train_obj.head()


,GENDER,AGE,CHARGE_CITY_CD,CONTACT_CITY_CD,LAST_A_CCONTACT_DT,LAST_A_ISSUE_DT,LAST_B_ISSUE_DT,APC_1ST_AGE,INSD_1ST_AGE,IF_2ND_GEN_IND,...,IF_ADD_INSD_IND,CUST_9_SEGMENTS_CD,FINANCETOOLS_A,FINANCETOOLS_B,FINANCETOOLS_C,FINANCETOOLS_D,FINANCETOOLS_E,FINANCETOOLS_F,FINANCETOOLS_G,Y1
0,1,2,0,0,1,0,0,2,2,0,...,0,2,1,0,0,0,0,0,0,0
1,1,2,0,0,1,0,0,2,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,2,0,0,1,1,0,2,0,1,...,0,0,1,0,0,0,0,0,0,0
3,1,2,0,0,1,0,0,2,0,0,...,0,2,1,0,0,0,0,0,0,0
4,1,2,0,0,0,0,0,2,2,1,...,0,2,1,0,0,0,0,0,0,0


### Numeric --> Fillna(mean)

In [25]:
train_num = train.select_dtypes(include=['float64','int'])
test_num = test.select_dtypes(include=['float64','int'])

Don't forget that some data is objective bt present in numeric type.  
We could detect them by observing unique value

In [26]:
for col in train_num.columns:
    if train_num[col].nunique()<=10: #if unique value less than 10,treat it as objective variable
        train_num[col].fillna(train_num[col].mode()[0],inplace=True)
    else:
        train_num[col].fillna(round(train_num[col].mean()),inplace=True) #fillna with mean

for col in test_num.columns:
    if test_num[col].nunique()<=10: #if unique value less than 10,treat it as objective variable
        test_num[col].fillna(test_num[col].mode()[0],inplace=True)
    else:
        test_num[col].fillna(round(test_num[col].mean()),inplace=True) #fillna with mean



In [28]:
train_num.head()


,CUS_ID,EDUCATION_CD,MARRIAGE_CD,L1YR_A_ISSUE_CNT,L1YR_B_ISSUE_CNT,CHANNEL_A_POL_CNT,CHANNEL_B_POL_CNT,OCCUPATION_CLASS_CD,APC_CNT,INSD_CNT,...,OUTPATIENT_SURGERY_AMT,INPATIENT_SURGERY_AMT,PAY_LIMIT_MED_MISC_AMT,FIRST_CANCER_AMT,ILL_ACCELERATION_AMT,ILL_ADDITIONAL_AMT,LONG_TERM_CARE_AMT,MONTHLY_CARE_AMT,LIFE_INSD_CNT,L1YR_GROSS_PRE_AMT
0,3418,3.0,0.0,0,0,0,0,1.0,0,0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.015625,0.000174
1,4302,3.0,0.0,0,0,0,0,1.0,0,0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.015625,0.008724
2,5545,1.0,0.0,1,0,1,0,1.0,1,0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.031250,0.005359
3,7207,3.0,0.0,0,0,0,0,1.0,0,0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.015625,0.000000
4,7213,1.0,0.0,0,0,1,0,1.0,1,0,...,0.0,0.0,0.0,0.04717,0.0,0.0,0.0,0.0,0.062500,0.000603


In [29]:
train_clean = pd.concat([train_num , train_obj], axis = 1)
test_clean = pd.concat([test_num , test_obj], axis = 1)

train_clean.head()

,CUS_ID,EDUCATION_CD,MARRIAGE_CD,L1YR_A_ISSUE_CNT,L1YR_B_ISSUE_CNT,CHANNEL_A_POL_CNT,CHANNEL_B_POL_CNT,OCCUPATION_CLASS_CD,APC_CNT,INSD_CNT,...,IF_ADD_INSD_IND,CUST_9_SEGMENTS_CD,FINANCETOOLS_A,FINANCETOOLS_B,FINANCETOOLS_C,FINANCETOOLS_D,FINANCETOOLS_E,FINANCETOOLS_F,FINANCETOOLS_G,Y1
0,3418,3.0,0.0,0,0,0,0,1.0,0,0,...,0,2,1,0,0,0,0,0,0,0
1,4302,3.0,0.0,0,0,0,0,1.0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5545,1.0,0.0,1,0,1,0,1.0,1,0,...,0,0,1,0,0,0,0,0,0,0
3,7207,3.0,0.0,0,0,0,0,1.0,0,0,...,0,2,1,0,0,0,0,0,0,0
4,7213,1.0,0.0,0,0,1,0,1.0,1,0,...,0,2,1,0,0,0,0,0,0,0


### Export  CLEAN data for further Usage

In [34]:
import csv
train_clean.to_csv('/Users/liaoshousan/Desktop/Cathay_BigData-Competition/Data/train2.csv',index = None)
test_clean.to_csv('/Users/liaoshousan/Desktop/Cathay_BigData-Competition/Data/test2.csv',index = None)

### Execute XGBOOST

In [12]:
%%time
from sklearn.model_selection import train_test_split
X = train_clean.drop(columns=['CUS_ID','Y1'])
y = train_clean['Y1']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)

xgbc = XGBClassifier()
xgbc.fit(X_train, y_train)


print('The accuracy of eXtreme Gradient Boosting Classifier on testing set:', xgbc.score(X_test, y_test))


The accuracy of eXtreme Gradient Boosting Classifier on testing set: 0.98092
CPU times: user 1min, sys: 757 ms, total: 1min 1s
Wall time: 1min 2s


準確率0.9809 但是roc_auc socre呢

In [13]:
from sklearn.metrics import roc_auc_score
test_predict = xgbc.predict(X_test)
roc_auc_score(test_predict,y_test)

0.6155184829572732

Measure 1  
roc_auc score只有0.6155 顯然是很糟的模型

# Measure 2

In [14]:
def fillna_in_proportion(dataframe,colname):
    col_unique = dataframe[colname].dropna().unique()
    size = dataframe[colname].shape[0]
    attribute_p = list(dataframe[colname].value_counts()/dataframe[colname].dropna().shape[0])
    dataframe[colname].fillna(pd.Series(np.random.choice(a = col_unique,size = size , p=attribute_p)),inplace=True)



In [15]:
train_num = train.select_dtypes(include=['float64','int'])
train_obj = train_obj.apply(LabelEncoder().fit_transform)


In [16]:
# train_num --> 
for col in train_num.columns:
    if train_num[col].nunique()<=10: #if unique value less than 10,treat it as objective variable
        fillna_in_proportion(train_num,col)
    else:
        train_num[col].fillna(round(train_num[col].mean()),inplace=True) #fillna with mean

# train_obj --> filllna() in proportional
for col in train_obj.columns:
    fillna_in_proportion(train_obj,col)

/Users/liaoshousan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [17]:
train_clean = pd.concat([train_num , train_obj], axis = 1)
train_clean.head()

,CUS_ID,EDUCATION_CD,MARRIAGE_CD,L1YR_A_ISSUE_CNT,L1YR_B_ISSUE_CNT,CHANNEL_A_POL_CNT,CHANNEL_B_POL_CNT,OCCUPATION_CLASS_CD,APC_CNT,INSD_CNT,...,IF_ADD_INSD_IND,CUST_9_SEGMENTS_CD,FINANCETOOLS_A,FINANCETOOLS_B,FINANCETOOLS_C,FINANCETOOLS_D,FINANCETOOLS_E,FINANCETOOLS_F,FINANCETOOLS_G,Y1
0,3418,1.0,1.0,0,0,0,0,1.0,0,0,...,0,2,1,0,0,0,0,0,0,0
1,4302,3.0,0.0,0,0,0,0,1.0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5545,1.0,0.0,1,0,1,0,1.0,1,0,...,0,0,1,0,0,0,0,0,0,0
3,7207,3.0,0.0,0,0,0,0,1.0,0,0,...,0,2,1,0,0,0,0,0,0,0
4,7213,1.0,0.0,0,0,1,0,1.0,1,0,...,0,2,1,0,0,0,0,0,0,0


In [18]:
%%time
from sklearn.model_selection import train_test_split
X = train_clean.drop(columns=['CUS_ID','Y1'])
y = train_clean['Y1']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)

xgbc = XGBClassifier()
xgbc.fit(X_train, y_train)


print('The accuracy of eXtreme Gradient Boosting Classifier on testing set:', xgbc.score(X_test, y_test))


The accuracy of eXtreme Gradient Boosting Classifier on testing set: 0.98092
CPU times: user 1min 3s, sys: 816 ms, total: 1min 4s
Wall time: 1min 5s


### ROC_AUC Score進步到0.657了

In [19]:
from sklearn.metrics import roc_auc_score
test_predict = xgbc.predict(X_test)
roc_auc_score(test_predict,y_test)

0.6155184829572732

第一階段測試模型結束  
Measure 2 Roc_Auc score = 0.65  
在沒有調整參數＆無技巧地填值，Baseline = 0.65  